In [16]:
from dotenv import load_dotenv 
load_dotenv()
import os
from openai import OpenAI
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec


In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536, #the dimensions of pinecone should match with the opensource 
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [4]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'reviews': "Dr. Johnson's lectures are engaging and informative. She really knows how to break down complex concepts."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'reviews': 'Brilliant professor! His passion for coding is contagious. Challenging but rewarding classes.'},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Psychology',
  'stars': 3,
  'reviews': 'Interesting material, but sometimes the pace is too fast. Office hours are helpful.'},
 {'professor': 'Prof. Robert Taylor',
  'subject': 'History',
  'stars': 4,
  'reviews': 'Prof. Taylor brings history to life with his storytelling. Assignments can be tough but fair.'},
 {'professor': 'Dr. Lisa Martinez',
  'subject': 'Chemistry',
  'stars': 5,
  'reviews': "Best chemistry professor I've had. Clear explanations and fun lab experiments."},
 {'professor': 'Prof. David Wilson',
  'subject': 'Mathematics',
  'st

In [18]:
processed_data = []
client = Openai()
# client = genai()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
        # input=review['review'], model= genai.GenerativeModel('gemini-1.5-pro')
    )
    # response = genai.embeddings.create(
    #     input=review['review'], 
    #     model="gemini-1.5-flash"  # Replace with the correct model name if needed
    # )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


NameError: name 'Openai' is not defined

In [19]:
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 25 Aug 2024 04:42:46 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '41', 'x-pinecone-request-id': '970815574757713726', 'x-envoy-upstream-service-time': '42', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [15]:
print(index.describe_index_stats())


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
